In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt 
from tqdm.auto import tqdm as progressbar
import time
%matplotlib qt5

## Setup

This is now implemented in the snapshot/MjpgSnapshotterExp class

In [2]:
import subprocess
access_str = r"pi@192.168.0.15"

def ssh_command(cmd):
    try:
        return subprocess.check_output([
                "ssh",
                access_str,
                cmd
            ]).decode()
    except subprocess.CalledProcessError:
        time.sleep(0.5)
        return subprocess.check_output([
                "ssh",
                access_str,
                cmd
            ]).decode()


print(ssh_command("v4l2-ctl --all").split("Flags:")[-1])
print(ssh_command("v4l2-ctl -cexposure_auto=1")[-500:])

 
Streaming Parameters Video Capture:
	Capabilities     : timeperframe
	Frames per second: 30.000 (30/1)
	Read buffers     : 0
                     brightness 0x00980900 (int)    : min=0 max=255 step=1 default=128 value=128
                       contrast 0x00980901 (int)    : min=0 max=255 step=1 default=32 value=32
                     saturation 0x00980902 (int)    : min=0 max=255 step=1 default=32 value=32
 white_balance_temperature_auto 0x0098090c (bool)   : default=1 value=0
                           gain 0x00980913 (int)    : min=0 max=255 step=1 default=64 value=2
           power_line_frequency 0x00980918 (menu)   : min=0 max=2 default=2 value=2
      white_balance_temperature 0x0098091a (int)    : min=0 max=10000 step=10 default=4000 value=0
                      sharpness 0x0098091b (int)    : min=0 max=255 step=1 default=24 value=64
         backlight_compensation 0x0098091c (int)    : min=0 max=1 step=1 default=0 value=0
                  exposure_auto 0x009a0901 (menu)  

In [14]:
def set_exposure(value):
    return ssh_command(f"v4l2-ctl -cexposure_absolute={value}")

def set_temp(value):
    return ssh_command(f"v4l2-ctl -cwhite_balance_temperature={value}")

def set_gain(value):
    return ssh_command(f"v4l2-ctl -cgain={value}")

set_exposure(166)
ssh_command(f"v4l2-ctl -csaturation=32")
ssh_command(f"v4l2-ctl -cwhite_balance_temperature_auto=0")
ssh_command(f"v4l2-ctl -cwhite_balance_temperature=0")
ssh_command(f"v4l2-ctl -csharpness=24")

''

In [4]:
import sys
import time
sys.path.append(r"../ender")
from snapshot import MjpgSnapshotter


snap = MjpgSnapshotter("http://192.168.0.15/webcam/")

## Exposure sweep

In [5]:
images=[]
exposure_range = range(1,200,5)
set_gain(2)
for i in progressbar(exposure_range):
    set_exposure(i)
    ret, im = snap.read()
    time.sleep(0.5)
    if ret:
        images.append(im)

100%|██████████| 40/40 [00:41<00:00,  1.04s/it]


In [ ]:
fig, axs = plt.subplots(2)
axs[0].imshow(images[6])
axs[1].imshow(images[-1])

### Plot "reddest" pixels X exposure

In [6]:
red = np.array([ [ [ [1],[-0.5],[-0.5] ] ] ])
white = np.array([ [ [ [1],[1],[1] ] ] ])
white=white/np.linalg.norm(white)

In [7]:
I = np.stack(images)
I = I / 255 / (3**(1/3))
red_contrast = I@red
white_value = I@white

In [8]:
plt.figure()
plt.plot(np.max(red_contrast,axis=(1,2)))
plt.plot(np.max(white_value, axis=(1,2)) )
plt.show()
print(np.argmax(np.max(red_contrast, axis=(1,2)) ))

11


In [9]:
ids=[]
for img in np.linalg.norm(I, axis=-1):
    ids.append( np.unravel_index(img.flatten().argmax(), img.shape) )
max_norm_vecs = [ I[n, i[0], i[1]] for n, i in enumerate(ids) ]

### Plot brighest pixel's color X exposure

In [11]:
x = np.stack(max_norm_vecs) * 255 * 3**(1/3)
plt.plot(exposure_range, x[:,0], label="r", color ="red")
plt.plot(exposure_range, x[:,1], label="g", color ="green")
plt.plot(exposure_range, x[:,2], label="b", color ="blue")
plt.xlabel("exposure")
plt.legend()

### Plot hist X exposure

In [ ]:
I.shape

In [ ]:
fig, axs = plt.subplots(len(exposure_range)//4, 4)

for img, ax in zip(I, axs.flatten()):
    bnw = cv2.cvtColor((img * 255 * (3**(1/3))).astype(np.uint8), cv2.COLOR_RGB2GRAY)
    ax.hist(bnw[bnw > 30].flatten(), bins=20)
    

## Gain Sweep

In [ ]:
images_gain=[]
gain_range = range(5,255,8)
set_exposure(35)
for i in progressbar(gain_range):
    set_exposure(i)
    ret, im = snap.read()
    time.sleep(0.5)
    if ret:
        images_gain.append(im)

### Plot brighest pixel's color X gain

In [ ]:
I = np.stack(images_gain)
I = I / 255 / (3**(1/3))
red_contrast = I@red
white_value = I@white
ids=[]
for img in np.linalg.norm(I, axis=-1):
    ids.append( np.unravel_index(img.flatten().argmax(), img.shape) )
max_norm_vecs = [ I[n, i[0], i[1]] for n, i in enumerate(ids) ]

In [ ]:
x = np.stack(max_norm_vecs) * 255 * 3**(1/3)
plt.plot(gain_range, x[:,0], label="r", color ="red")
plt.plot(gain_range, x[:,1], label="g", color ="green")
plt.plot(gain_range, x[:,2], label="b", color ="blue")
plt.xlabel("gain")
plt.legend()